# Ch `02`: Concept `09`

## Using Queues

If you have a lot of training data, you probably don't want to load it all into memory at once. The QueueRunner in TensorFlow is a tool to efficiently employ a queue data-structure in a multi-threaded way.

In [1]:
import tensorflow as tf
import numpy as np

We will be running multiple threads, so let's figure out the number of CPUs:

In [2]:
import multiprocessing
NUM_THREADS = multiprocessing.cpu_count()

Generate some fake data to work with:

In [3]:
xs = np.random.randn(100, 3)
ys = np.random.randint(0, 2, size=100)

Here's a couple concrete examples of our data:

In [4]:
xs_and_ys = zip(xs, ys)
for _ in range(5):
    x, y = next(xs_and_ys)
    print('Input {}  --->  Output {}'.format(x, y))

Input [ 1.46034759  0.71462742  0.73288402]  --->  Output 0
Input [ 1.1537654  -0.09128405  0.08036941]  --->  Output 1
Input [-0.61164559 -0.19188485  0.06064167]  --->  Output 0
Input [ 0.1007337   0.34815357  0.24346031]  --->  Output 0
Input [-1.25581117  1.44738085  1.15035257]  --->  Output 0


Define a queue:

In [5]:
queue = tf.FIFOQueue(capacity=1000, dtypes=[tf.float32, tf.int32])

Set up the enqueue and dequeue ops:

In [6]:
enqueue_op = queue.enqueue_many([xs, ys])
x_op, y_op = queue.dequeue()

Define a QueueRunner:

In [7]:
qr = tf.train.QueueRunner(queue, [enqueue_op] * 4)

Now that all variables and ops have been defined, let's get started with a session:

In [8]:
sess = tf.InteractiveSession()

Create threads for the QueueRunner:

In [9]:
coord = tf.train.Coordinator()
enqueue_threads = qr.create_threads(sess, coord=coord, start=True)

Test out dequeueing:

In [10]:
for _ in range(100):
    if coord.should_stop():
        break
    x, y = sess.run([x_op, y_op])
    print(x, y)
coord.request_stop()
coord.join(enqueue_threads)

[ 1.46034753  0.71462744  0.73288405] 0
[ 1.15376544 -0.09128405  0.08036941] 1
[-0.61164558 -0.19188486  0.06064167] 0
[ 0.1007337   0.34815356  0.24346031] 0
[-1.25581121  1.4473809   1.1503526 ] 0
[ 0.60369009 -0.87942719 -1.37121975] 1
[ 1.30641925  1.55316997  1.01789773] 0
[ 0.0575242   0.59463078  0.47600508] 1
[-1.22782397 -0.86792755  1.37459588] 1
[-0.27896652  0.51645088  1.36873603] 0
[-0.34542757  0.79360306  0.32000065] 0
[-0.46792462 -0.31817994  0.91739392] 0
[ 0.24787657  0.83848852  1.16125166] 0
[-0.46220389 -0.09412029 -0.9981451 ] 1
[ 0.06739734 -1.08405316 -0.3582162 ] 1
[-1.2644819  -0.27479929  1.15882337] 1
[-0.68015367 -0.10199564  1.4274267 ] 0
[-0.48884565 -0.39484504  0.1496018 ] 1
[ 1.48414564 -0.43943462 -0.12646018] 0
[ 0.49450573  0.42091215 -0.17693481] 0
[ 0.02265234  0.99832052  0.26808155] 1
[-0.94086462  1.67000341  0.92434174] 1
[-0.50961769 -0.39044595 -0.5737586 ] 0
[-0.95702702  0.61196166 -0.86487901] 1
[-0.6125344  -0.30916786 -1.06602347] 1
